In [ ]:
import numpy as np
import geopandas as gpd
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon, Polygon
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from scipy.ndimage import distance_transform_edt
import random
from matplotlib.colors import ListedColormap

In [ ]:
num_districts = 27
tolerance = 50000  # Adjust as needed (not used in Voronoi, but kept for reference)

print("Total population within Florida:", population_masked.sum())
target_pop = population_masked.sum() / num_districts
print("Target population per district:", target_pop)

# Step 1: Identify Valid Pixels (within Florida)
valid_pixels = np.argwhere(population_masked > 0)

if len(valid_pixels) < num_districts:
    raise ValueError("Not enough valid pixels to assign to districts.")

# Step 2: Randomly Select 27 Seed Pixels
seed_indices = random.sample(range(valid_pixels.shape[0]), num_districts)
seed_coords = valid_pixels[seed_indices]

# Step 3: Create an Empty District Map
# Initialize with -1 for ocean pixels
district_map = -1 * np.ones((height, width), dtype=int)

# Step 4: Assign Each Seed Pixel to a Unique District
for d_id, (r, c) in enumerate(seed_coords):
    district_map[r, c] = d_id

# Step 5: Compute Voronoi Tessellation
# Create a boolean array where seed pixels are True
seed_mask = np.zeros((height, width), dtype=bool)
seed_mask[seed_coords[:, 0], seed_coords[:, 1]] = True

# Use distance_transform_edt to find the nearest seed pixel for each pixel
# The function returns distances and indices of the nearest True pixel
distance, indices = distance_transform_edt(~seed_mask, return_indices=True)

# Assign each pixel to the nearest seed's district ID
nearest_row = indices[0]
nearest_col = indices[1]
nearest_district = district_map[nearest_row, nearest_col]

# Update the district map for all valid pixels
district_map[population_masked > 0] = nearest_district[population_masked > 0]

# Pixels outside Florida remain -1 (black)

In [ ]:
# Step 1: Create a Custom Colormap
# - Districts: Use a colormap with enough distinct colors
# - Ocean Pixels: Black
district_colors = plt.get_cmap('tab20', num_districts).colors[:num_districts]
colors = list(district_colors) + [(0, 0, 0, 1)]  # Adding black for ocean
custom_cmap = ListedColormap(colors)

# Step 2: Adjust district_map to Shift -1 to the Last Index (for black color)
district_map_shifted = district_map.copy()
district_map_shifted[district_map_shifted == -1] = num_districts  # Assign -1 to black

# Step 3: Plot the District Map
plt.figure(figsize=(12, 12))
im = plt.imshow(
    district_map_shifted,
    cmap=custom_cmap,
    interpolation='nearest',
    origin='upper',
    extent=(minx, maxx, miny, maxy)
)
# cbar = plt.colorbar(im, ticks=range(num_districts + 1))
# cbar.ax.set_yticklabels([str(i) for i in range(num_districts)] + ['Outside Florida'])
plt.title('27 Districts Generated via Voronoi Tessellation (Black = Outside Florida)')
plt.grid(False)
plt.xticks([])
plt.yticks([])
plt.axis('equal')
plt.show()

In [ ]:
# --------------------------------------
# Part 4: Calculate and Plot Population and Vote Percentages
# --------------------------------------

# Initialize lists to store data for each district
district_ids = range(num_districts)
total_population = []
total_blue = []
total_red = []
percentage_blue = []
percentage_red = []

for d_id in district_ids:
    # Create a mask for the current district
    mask = (district_map == d_id)
    
    # Calculate total population for the district
    pop = population_masked[mask].sum()
    total_population.append(pop)
    
    # Calculate total blue and red votes for the district
    blue = blue_band[mask].sum()
    red = red_band[mask].sum()
    total_blue.append(blue)
    total_red.append(red)
    
    # Calculate percentage of blue and red votes
    if pop > 0:
        percentage_blue.append((blue / pop) * 100)
        percentage_red.append((red / pop) * 100)
    else:
        percentage_blue.append(0)
        percentage_red.append(0)

# Create a DataFrame to organize the data
data = {
    'District': district_ids,
    'Total Population': total_population,
    'Blue Votes': total_blue,
    'Red Votes': total_red,
    'Percentage Blue': percentage_blue,
    'Percentage Red': percentage_red
}

df = pd.DataFrame(data)

# Display the DataFrame (optional)
# print(df)

# --------------------------------------
# Plot: Total Population and Votes per District
# --------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(28, 7))  # 1 row, 2 columns

# Plot 1: Total Population per District
axes[0].bar(df['District'], df['Total Population'], color='lightgreen')
axes[0].set_xlabel('District ID', fontsize=14)
axes[0].set_ylabel('Total Population', fontsize=14)
axes[0].set_title('Total Population per District', fontsize=16)
axes[0].set_xticks(df['District'])
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Plot 2: Percentage of Blue vs Red Votes per District
width = 0.6  # Width of the bars

# Plot Blue Votes
axes[1].bar(df['District'], df['Percentage Blue'], width, label='Blue', color='skyblue')

# Plot Red Votes on top of Blue Votes
axes[1].bar(df['District'], df['Percentage Red'], width, bottom=df['Percentage Blue'], label='Red', color='pink')

axes[1].set_xlabel('District ID', fontsize=14)
axes[1].set_ylabel('Percentage of Votes (%)', fontsize=14)
axes[1].set_title('Percentage of Blue vs Red Votes per District', fontsize=16)
axes[1].set_xticks(df['District'])
axes[1].legend(fontsize=12)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
# --------------------------------------
# Part 4: Calculate and Plot Population and Vote Percentages
# --------------------------------------

# Initialize lists to store data for each district
district_ids = range(num_districts)
total_population = []
total_blue = []
total_red = []
percentage_blue = []
percentage_red = []

for d_id in district_ids:
    # Create a mask for the current district
    mask = (district_map == d_id)
    
    # Calculate total population for the district
    pop = population_masked[mask].sum()
    total_population.append(pop)
    
    # Calculate total blue and red votes for the district
    blue = blue_band[mask].sum()
    red = red_band[mask].sum()
    total_blue.append(blue)
    total_red.append(red)
    
    # Calculate percentage of blue and red votes
    if pop > 0:
        percentage_blue.append((blue / pop) * 100)
        percentage_red.append((red / pop) * 100)
    else:
        percentage_blue.append(0)
        percentage_red.append(0)

# Create a DataFrame to organize the data
data = {
    'District': district_ids,
    'Total Population': total_population,
    'Blue Votes': total_blue,
    'Red Votes': total_red,
    'Percentage Blue': percentage_blue,
    'Percentage Red': percentage_red
}

df = pd.DataFrame(data)

# Display the DataFrame (optional)
# print(df)

# --------------------------------------
# Plot: Total Population and Votes per District
# --------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(28, 7))  # 1 row, 2 columns

# Plot 1: Total Population per District
axes[0].bar(df['District'], df['Total Population'], color='lightgreen')
axes[0].set_xlabel('District ID', fontsize=14)
axes[0].set_ylabel('Total Population', fontsize=14)
axes[0].set_title('Total Population per District', fontsize=16)
axes[0].set_xticks(df['District'])
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Plot 2: Percentage of Blue vs Red Votes per District
width = 0.6  # Width of the bars

# Plot Blue Votes
axes[1].bar(df['District'], df['Percentage Blue'], width, label='Blue', color='skyblue')

# Plot Red Votes on top of Blue Votes
axes[1].bar(df['District'], df['Percentage Red'], width, bottom=df['Percentage Blue'], label='Red', color='pink')

axes[1].set_xlabel('District ID', fontsize=14)
axes[1].set_ylabel('Percentage of Votes (%)', fontsize=14)
axes[1].set_title('Percentage of Blue vs Red Votes per District', fontsize=16)
axes[1].set_xticks(df['District'])
axes[1].legend(fontsize=12)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()
